# Análise de dados de RH

Notebook para análise da probabilidade de desligamento dos funcionários ativos de uma empresa fictícia utilizando o algorítimo de árvores randômicas. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Importando bibliotecas do SKLearn

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

## Faz a leitura do DATASET e exibe os detalhes da base
O dataset tem dados de colaboradores de uma empresa fictícia para análises voltadas ao entendimento da saída desses colaboradores da companhia, onde podem ser consideradas características profissionais, sociais e pessoais dos mesmos. 
Temos como objetivo principal nesse estudo, calcular a probabilidade de saída dos demais colaboradores ativos da CIA, com base no perfil dos desligados.

In [ ]:
df = pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
df.describe()

## Organização dos dados
* FEATURES -> características selecionadas para identificação do perfil dos colaboradores
* LABEL -> classe de dados para análise, nesse caso a relação se o colaborados teve ou não desligamento
* INDEXES -> índices para cruzamento posterior das probabilidades com os colaboradores

In [ ]:
features = df.iloc[:, df.columns != 'Attrition']
labels = df.iloc[:,1]
indexes = df.index.values

## Limpeza e Separação dos dados para treino e teste
Nos passos a seguir serão feitas as seguintes tarefas:
* limpeza e harmonização do dataset de features
* tratamento de variáveis categóricas e ordinais
* separação das bases pra teste e treino

In [ ]:
features = features.drop(
    columns=['MonthlyRate','EmployeeCount','EmployeeNumber','DailyRate','HourlyRate','Over18','OverTime'
    ,'StockOptionLevel','YearsWithCurrManager','TotalWorkingYears','TrainingTimesLastYear'], axis=1)
features.columns

In [ ]:
transformer = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,2,4,5,6,7,8,9,10,11,12,16,17,21])], remainder="passthrough")
features = transformer.fit_transform(features)

In [ ]:
le_labels = LabelEncoder()
labels = le_labels.fit_transform(labels)

In [ ]:
trainFeatures, testFeatures, trainIndexes, targetIndexes = train_test_split(features, indexes, test_size=0.30, random_state=42)
trainLabels, targetLabels = labels[trainIndexes], labels[targetIndexes]

## Treinamento do Model
Abaixo é feita a configuração do algorítimo de Nayve Bayes e a o modelo de treino é submetido para a classificação dos dados.

In [ ]:
model = RandomForestClassifier(n_estimators = 200, random_state = 0, criterion='entropy', max_depth=None, min_samples_split=5)
model.fit(trainFeatures, trainLabels)

## Teste e Avaliação 
Abaixo é feito o teste das predições considerando a base de teste e também é calculada a acurária do algorítimo de Árvore Randômica comparando o resultado das predições com a base de comparação chamada de targetLabels

In [ ]:
predictions = model.predict(testFeatures)

In [ ]:
precision = accuracy_score(targetLabels, predictions)
matrix = confusion_matrix(targetLabels, predictions)
print('Acurácia do algorítmo ->', round(precision, 2) * 100, '%') 

## Classificação de risco dos Ativos
Nesta parte, será feita a plotagem do risco de saída dos colaboradores da massa de teste, dessa maneira, o setor de recursos humanos poderia escolher os colaboradores com maior probabilidade de saída para então pensar em atitudes de retenção dos mesmos na CIA, tendo em vista que todo o dataset já considera apenas os colabores com notas iguais ou acima de excelente na avaliação de desempenho

In [ ]:
test_probabilities = model.predict_proba(testFeatures)
train_probabilities = model.predict_proba(trainFeatures)

In [ ]:
df_new = df.copy()
df_new.loc[targetIndexes,'probability'] = test_probabilities[:,1]
df_new.loc[trainIndexes,'probability'] = train_probabilities[:,1]

In [ ]:
no_attrition = df_new['Attrition']=='No'
df_no_attrition = df_new[no_attrition]

df_no_attrition.head(10)

## Salvando base com probabilidade de desligamento

In [ ]:
df_no_attrition.to_csv('HR-Employee-Probability-Attrition.csv', index=False)